# Todo - will remove this section before submittion 
* change code using without for loops

Kavya github link
https://github.com/KavyaOS/Cryptocurrency_Analysis_PricePrediction.git

Errors while running Kavya nottebook:
1.  ModuleNotFoundError: No module named 'plotly'
sol: 
pip install plotly

2. ModuleNotFoundError: No module named 'keras'

sol:
pip install keras

3. ImportError: Keras requires TensorFlow 2.2 or higher. Install TensorFlow via `pip install tensorflow`
pip install tensorflow








Prof notes - 
Given that everyone going different things for the project I want you to add a short  description in your notebook. There are up to six things that I want.

First if you are working on a team then include the names and Red IDs of both team members at the top of the readme file. Just have one person submit the project so I don’t end up grading the same project twice.

Second a brief description of what the project does. What are the goals of your assignment? Did you succeed? What did you learn. In the assignments I knew what you were to do. So if you just had code and tables I could figure things out. In your project you need to explain what your code is doing and what the output of your code means. In Jupyter notebooks you can use markdown cells to contain text. Use the markdown cells to to explain your code and results as they are easier to read then comments in code. Use the markdown cells to add headers (Introduction, How to Run, Results, etc) to give the notebook some structure.  

Third any special instructions that I need to run the project. 

Fourth if you use any third party libraries list them and give the pip command needed to install them. A sentence or two on what the library does for you. If your idea comes from a site like Kaggle include a link to it. 

Fifth a list know issues with the project. If a feature is not working let me know. If a feature is not working I will grade it more severe than if you do not tell me.

Six do not upload large data files. If your total upload to the course portal is 5 MB you will be fine. If people start uploading large files to the course portal the server will crash causing you and other people in the class to panic. (I need to rewrite that server.) If you data file puts you over that limit add a command in your jupyter notebook that will  download your data file from its source. The following command in a code cell on a Unix machine will download covid data at the given https address to the local file covid.csv.


!curl -o covid.csv https://usafactsstatic.blob.core.windows.net/public/data/covid-19/covid_confirmed_usafacts.csv 


If your code takes a long time to run on the original data set then I would like a smaller dataset to run while grading. If the file is small enough it can be combined with your notebook and up loaded. If not provide a method to download the file: dropbox link, Google drive link, etc. If you are not able to do this contact me to make other arrangements. Always include a link to access the original data set file.


As with assignments and the exam you will submit the project to the course portal.

#  Big Data CS649 Project -  Kavya and Sachin Kumar (RedIds - 825893660 and 823431551)

## Info

### Objective :
- Analysing the trend of crypto currency for the last few years.
- Cryptocurrency price Prediction using Deep learning. (for example ML models - Linear Regression,  K-Nearest Neighbors.)
- Analysis of social media on the crypto prize. Sentiments using few keywords in the tweets and relate with the currency price.  Positive, negative or neutral emotions.
- Our main focus would be on Twitter and Bit-coin. We can explore more currencies and platforms if time permits. 


### Datasets :
- Tweets dataset : https://www.kaggle.com/alaix14/bitcoin-tweets-20160101-to-20190329
    - Contains all the bitcoin tweets
    - date range is from 2016-Jan-01 to 2019-March-29
    
### Special Library/Packages installed : 
- TextBlob
    - pip install -U textblob
    - https://textblob.readthedocs.io/en/dev/install.html
- Language detector and translator
    - pip install google_trans_new
    - https://github.com/lushan88a/google_trans_new
- pip install pandas-profiling
- conda install graphviz
- #todo : might need to delete below packages
- pip install atoti 
- pip install atoti-aws



### Approach :


### Chalenges :
 - Not able to install few packages on windows like whatthelang
    - Language Prediction
    - https://github.com/indix/whatthelang
    - pip install -r requirements.txt
    - pip install whatthelang
 - Google translator stopped translating the tweets after certain time with below error.
    - Error - "Our systems have detected unusual traffic from your computer network"

# Running instruction

#### Packages that needs to be installed before running this notebook. Uncomment below lines once to install it and then comment it again.


In [1]:
# ! pip install -U textblob
# ! pip install google_trans_new

In [2]:
# download english stopwords, this might take little time
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sk105659\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

#### Few configurations before running the notebook


In [3]:
######### Dataset input file path ###########################################################################
bitcoin_tweets_dataset_filepath = '../input_datasets/tweets.csv'   # Input dataset which needt to downloaded from Kaggle as given above.
  

################## Number of records to be processed from the dataset input file ###########################
# this is important as the dataset is very huge and we have many bottleneck services like - 
# 1) google lan detector
# 2) google lan translatoin, 
# 3) removing english verbose words from tweet text using stopwords from nltk.corpus
# 4) finding sentiments out of english words using TextBlob

no_of_records = 99999   # This seems to be a decent number as per our anlysis - output & performance
# no_of_records = 1000  


######### Goggle translation flag to enable english translation or not#################################
#   todo : detecotor reached to limit after certain hits
# if get this error : google_new_transError: 429 (Too Many Requests) from TTS API. then please change below flag from 0 to 1. 
# Doing so, it will not convert non-english to english lan and therefore the sentiments calucations are not 100% accurate.

# Its a common issue for all free user : https://github.com/lushan88a/google_trans_new/issues/28
# we have tried many options like giving differnt proxies and making connection using private VPN and keep switching the VPN tunnels (specially to Mexico worked) to get the desired ouput for 100000 records.

is_eng_translator_limit_reached = 0   # 0 if limit not recached, 1 if limit reached, 
is_detect_lang_required = 0           # 0 if not required , 1 if required. Turning this off is good as sometime it slows or reach to thresold limit faster than translation itself


# Also note, once the translation is done, the output would be saved on the disk. 
# and so for all the future runs, no translation is required, ranslated data would be read from the saved disk.

processed_bitcoin_tweets_dataset_filepath = './datasets/tweets_processed_v50s_eng.csv'  # Name of the file after google translation work is done once. 
# processed_bitcoin_tweets_dataset_filepath = '../input_datasets/tweets_processed_test10_eng.csv'



In [4]:
import numpy as np 
import pandas as pd 
from textblob import TextBlob
import matplotlib.pyplot as plt
from pandas.tseries.offsets import BDay


def read_and_clean_input_file(input_file):
    
    # todo : using small dataset for the devlopment purpose
    bitcoin_tweets_dataset_df_raw = pd.read_csv(
        input_file, 
        sep=';',
        nrows=no_of_records,
#         dtype={'text': str},
        converters={'text': str},
        usecols=['timestamp', 'text'],                            
        index_col=["timestamp"])
    
#     display(bitcoin_tweets_dataset_df_raw)
#     display(bitcoin_tweets_dataset_df_raw.info())

    
    # Strip time information from the column 'timestamp' if any
#     bitcoin_tweets_dataset_df_raw['timestamp'] = pd.to_datetime(bitcoin_tweets_dataset_df_raw['timestamp']).dt.date
    bitcoin_tweets_dataset_df_raw.index = pd.Series(pd.to_datetime(bitcoin_tweets_dataset_df_raw.index)).dt.date
    
#     display(bitcoin_tweets_dataset_df_raw)
#     display(bitcoin_tweets_dataset_df_raw.info())
    
    # df['timestamp'] = df['timestamp'].astype('datetime64[ns]')
    return bitcoin_tweets_dataset_df_raw


def filter_by_daterange(df):
    start_date = pd.to_datetime("2017-03-01").date()
    end_date = pd.to_datetime("2019-03-29").date()
    filtered_date_range = (df.index >= start_date) & (df.index <= end_date)
    df = df[filtered_date_range]
    ### Filter only Business day
    is_business_day = BDay().is_on_offset
    filtered_business_days = pd.to_datetime(df.index).map(is_business_day)
    df[filtered_business_days]
    return df

In [5]:
########### Below function is just for anlysing the dataset #####################################################
    
def analysing_dataset(df):
    display(df)
    display(df.info())
    display(df.index.min())
    display(df.index.max())
#     finding maximum and minimum date
    

########### Testing block #####################################################################################################
# bitcoin_tweets_dataset_df_cleaned = read_and_clean_input_file(bitcoin_tweets_dataset_filepath)
# analysing_dataset(bitcoin_tweets_dataset_df_cleaned)
# bitcoin_tweets_dataset_df_cleaned = filter_by_daterange(bitcoin_tweets_dataset_df_cleaned)
# analysing_dataset(bitcoin_tweets_dataset_df_cleaned)

In [6]:
from google_trans_new import google_translator  

# translator = google_translator()
translator = google_translator(timeout=30)
# translator = google_translator(url_suffix="https",timeout=5,proxies={'https':'206.189.44.99:8080','http':'96.9.77.203:55667','https':'140.227.68.230:58888','https':'72.196.184.54:8081' })
detector = google_translator()  

def add_english_translated_tweets_to_df(df):
    english_translated_col = []
    df['text'] = df['text'].astype(str)
    for tweet in df['text']:
        if len(tweet)!=0:
            if is_detect_lang_required == 1 :
                print("detecting lan first")
                detect_lang = detector.detect(tweet)
                if (detect_lang != ['en', 'english']) :
                    english_translated_col.append(translatoror.translate(tweet,lang_tgt='en') )
                else:
                    english_translated_col.append(tweet)
            else:
                print("detecting lan not required")
                english_translated_col.append(translator.translate(tweet,lang_tgt='en') )
        else:
            english_translated_col.append(None)

    df['english_translated'] = english_translated_col
    return df

############  Testing block  ##########################################

# function to detect lan
# detector = google_translator()  
# text = "È appena uscito un nuovo video! LES CRYPTOMONN..."
# text = "Another text which is already in english, so should not be translated."
# detect_lang = 'translate all text'
# detect_lang = detector.detect(text)
# print(detect_lang)


# function to translate the text to english
# if (detect_lang != ['en', 'english']) :
#     print(translator.translate(text,lang_tgt='en') )
# print(translator.translate(text,lang_tgt='en') )

In [7]:

def read_clean_filter_translate_write_file(input_file):
    bitcoin_tweets_dataset_df_cleaned = read_and_clean_input_file(input_file)
    bitcoin_tweets_dataset_df_cleaned = filter_by_daterange(bitcoin_tweets_dataset_df_cleaned)
    
    if(is_eng_translator_limit_reached != 1):
        bitcoin_tweets_dataset_df_cleaned = add_english_translated_tweets_to_df(bitcoin_tweets_dataset_df_cleaned)
    else:        
        bitcoin_tweets_dataset_df_cleaned['english_translated'] = bitcoin_tweets_dataset_df_cleaned['text']

    ##### Write the processed ouput to disk. This is to avoid unnecssary english translation everytime we run the program
    bitcoin_tweets_dataset_df_cleaned.to_csv(index=True, sep=',' , path_or_buf=processed_bitcoin_tweets_dataset_filepath)
#     display(bitcoin_tweets_dataset_df_cleaned)
    
    
############  Testing block  ##########################################  
# read_clean_filter_translate_write_file(bitcoin_tweets_dataset_filepath)

In [8]:
# todo: Note the seperator delimeter. orginal file has ';' and processed file has ','. 
# Should be good across complete file

import os.path
from os import path
processed_bitcoin_tweets_dataset_filepath 

if path.exists(processed_bitcoin_tweets_dataset_filepath):
    print ("processed bitcoin_tweets dataset filep exist")
else:
    print ("processed bitcoin_tweets dataset file does not exist")
    read_clean_filter_translate_write_file(bitcoin_tweets_dataset_filepath)




processed bitcoin_tweets dataset filep exist


In [9]:
processed_bitcoin_tweets_dataset_df = pd.read_csv(
        processed_bitcoin_tweets_dataset_filepath, 
        sep=',',
        dtype= {'text': str, 'english_translated': str},
        # dtype= str,
        # converters={'english_translated': str},
        index_col=["timestamp"])
    
# Strip time information from the column 'timestamp' if any
# processed_bitcoin_tweets_dataset_df['timestamp'] = pd.to_datetime(processed_bitcoin_tweets_dataset_df['timestamp']).dt.date
processed_bitcoin_tweets_dataset_df.index = pd.Series(pd.to_datetime(processed_bitcoin_tweets_dataset_df.index)).dt.date
display(processed_bitcoin_tweets_dataset_df)


# Filter the dates between a range which is common to both dataframes - tweets dataframe and price dataframe
processed_bitcoin_tweets_dataset_df = filter_by_daterange(processed_bitcoin_tweets_dataset_df)
# processed_bitcoin_tweets_dataset_df = processed_bitcoin_tweets_dataset_df.sort_index()   -- no need to sort. that is just for verification and display purpose, if needed.

display(processed_bitcoin_tweets_dataset_df.sample(10))
print('min date is ',processed_bitcoin_tweets_dataset_df.index.min())
print('max date is ',processed_bitcoin_tweets_dataset_df.index.max())

,text,english_translated
timestamp,,
2019-03-12,#Bitcoin #Satoshi #crypto #blockchain #Airdrop...,#Bitcoin #Satoshi #crypto #blockchain #Airdrop...
2018-09-11,I didnt tether at $7300 #iamspartacus\n$btc $e...,I didnt tether at $7300 #iamspartacus\n$btc $e...
2019-03-21,#Bitcoin #Satoshi #crypto #blockchain #Airdrop...,#Bitcoin #Satoshi #crypto #blockchain #Airdrop...
2019-03-09,#Bitcoin #Satoshi #crypto #blockchain #Airdrop...,#Bitcoin #Satoshi #crypto #blockchain #Airdrop...
2019-03-12,#Bitcoin #crypto #Airdrop\nNew Airdrop #p2pb2b...,#Bitcoin #crypto #Airdrop\nNew Airdrop #p2pb2b...
...,...,...
2018-12-28,"How the ""Times"" have changed - great to see su...","How the ""Times"" have changed - great to see su..."
2019-03-08,My weekly #Bitcoin and #crypto report for Bitc...,My weekly #Bitcoin and #crypto report for Bitc...
2018-12-25,Why gift cards are scams:\n1) turns liquid cas...,Why gift cards are scams:\n1) turns liquid cas...


,text,english_translated
timestamp,,
2018-05-01,02/05/2018 - 00:00\n=========================...,02/05/2018 - 00:00 ==========================...
2019-01-20,#monday #HorseRacing #tips\nTip 1 Wolves 6:25 ...,#monday #HorseRacing #tips\nTip 1 Wolves 6:25 ...
2018-07-07,またイベントで歌うことになりました笑\nあのね、やっぱりね、エンタメで有名になりたいのよw\...,"I will sing at the event again Well, after al..."
2018-09-03,New post in Verified Crypto News: #MarketCap\n...,New post in Verified Crypto News: #MarketCap\n...
2018-12-27,Top 5 #cryptocurrencies \n Alert Time: 2018-12...,Top 5 #cryptocurrencies \n Alert Time: 2018-12...
2018-05-08,"Launch!!! Not later than October 2018, 20:00 (...","Launch!!! Not later than October 2018, 20:00 (..."
2018-09-07,"こんばんは！2018-09-07 20:00 レート情報\nBTC：707,236 (↓-1...",Good evening! 2018-09-07 20:00 Rate Informatio...
2017-05-25,#Bitcoin #thatBitcoinlife will provide everyth...,#Bitcoin #thatBitcoinlife will provide everyth...
2018-09-23,#LMA $LMA Trading competition https://www.coin...,#LMA $LMA Trading competition https://www.coin...


min date is  2017-05-24
max date is  2019-03-29


In [10]:
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')

""" Utility function to clean tweet text by removing special characters and links using regex"""
def clean_special_chars(text):
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+://\S+)", " ", text).split())


""" Utility function to clean tweet text and remove verbose english words using nltk stopwords"""
def clean_and_remove_english_verbose_words(text):
    text = clean_special_chars(text)
    forbidden_words = set(stopwords.words('english'))
    text = ' '.join(text.split('.'))
    text = re.sub('\/',' ',text)
    text = text.strip('\'"')
    text = re.sub(r'@([^\s]+)',r'\1',text)
    text = re.sub(r'\\',' ',text)
    text = text.lower()
    text = re.sub('[\s]+', ' ', text)
    text = re.sub(r'#([^\s]+)', r'\1', text)
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',text)
    text = re.sub(r'((http)\S+)','',text)
    text = re.sub(r'\s+', ' ', re.sub('[^A-Za-z]', ' ', text.strip().lower())).strip()
    text = re.sub(r'\W+', ' ', text.strip().lower()).strip()
    text = [word for word in text.split() if word not in forbidden_words]
    return ' '.join(text)


""" Utility function to classify sentiment of passed tweet using textblob's sentiment method """
def sentiment(text):
    # create TextBlob object of passed tweet text
    text_analysis = TextBlob(text)
    return text_analysis.sentiment.polarity



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sk105659\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from textblob import TextBlob

""" Utility function to classify sentiment of passed tweet using textblob's sentiment method """
def add_eng_keywords_and_sentiments(df):
    english_keywords_col = []
    sentiment_col = []
    for tweet in df['english_translated']:
        
        # If running stopwords on big data slows donw the speed, then we can skip stopwords logic for performance and only use the simple cleaning of special chars
        english_keywords = clean_and_remove_english_verbose_words(tweet)
        #   english_keywords = clean_special_chars(tweet)
        english_keywords_col.append(english_keywords)
        
        # Add corresponding sentiments
        sentiment_polarity = sentiment(tweet)
        sentiment_col.append(sentiment_polarity)
    
    df['english_keywords'] = english_keywords_col
    df['sentiment'] = sentiment_col
    return df


# processed_bitcoin_tweets_dataset_df['english_keywords'] = processed_bitcoin_tweets_dataset_df['english_translated'].apply(lambda text: clean_and_remove_english_verbose_words(text))
# processed_bitcoin_tweets_dataset_df['sentiment'] = processed_bitcoin_tweets_dataset_df['english_keywords'].apply(lambda text: sentiment(text)) # new column of sentiment
# processed_bitcoin_tweets_dataset_df.drop(columns='english_translated', inplace=True)

processed_bitcoin_tweets_dataset_df = add_eng_keywords_and_sentiments(processed_bitcoin_tweets_dataset_df)
processed_bitcoin_tweets_dataset_df.sample(10)


In [ ]:
# # Filter the dates between a range which is common to both dataframes - tweets dataframe and price dataframe
# processed_bitcoin_tweets_dataset_df = filter_by_daterange(processed_bitcoin_tweets_dataset_df)
# # processed_bitcoin_tweets_dataset_df = processed_bitcoin_tweets_dataset_df.sort_index()   -- no need to sort. that is just for verification and display purpose, if needed.
# processed_bitcoin_tweets_dataset_df.sample(10)

In [ ]:
# considering the average tweet sentiment of the day.
# df = df.groupby(df.index).agg(lambda x: x.value_counts().index[0])
#  todo: played with first, max, min, mean. i think mean is the best one for this.
bitcoin_tweets_sentiment_df = processed_bitcoin_tweets_dataset_df.groupby(processed_bitcoin_tweets_dataset_df.index).min()
bitcoin_tweets_sentiment_df

In [ ]:
###################### from other notebook #######################
%store -r big_frame
big_frame

In [ ]:
bitcoin_price_df = big_frame.query("coin_name == 'Bitcoin'")
# todo : remove warning.. ... 
df1 = bitcoin_price_df['Date']
x2  = bitcoin_price_df.loc[:,['Date']]
x2
bitcoin_price_df['Date'] = pd.to_datetime(df1).dt.date
# bitcoin_price_df['Date'] = pd.to_datetime(x2).dt.date
# bitcoin_price_df['Date'] = pd.to_datetime(x2.stack())

bitcoin_price_df = bitcoin_price_df.set_index(['Date'])
display(bitcoin_price_df)
# display(bitcoin_price_df.info())

# Filter the dates between a range. This is common to both dataframes - tweets dataframe and price dataframe
bitcoin_price_df = filter_by_daterange(bitcoin_price_df)
# bitcoin_price_df = bitcoin_price_df['Price'].to_frame()
bitcoin_price_df = bitcoin_price_df[['Price', 'Change']]
display(bitcoin_price_df)
display(type(bitcoin_price_df))
# display(bitcoin_price_df.info())

In [ ]:
def merge_bitcoin_price_and_tweets_data(bitcoin_price_df, bitcoin_tweets_sentiment_df):
    #Combine two dataframes based on time.
    df = pd.merge(bitcoin_price_df, bitcoin_tweets_sentiment_df, left_index=True, right_index=True, how='inner')
    return df

def make_sentiment_column_categorical(bitcoin_price_sentiment):
    # Adding 'Sentiment categorical' column.
    bitcoin_price_sentiment['sentiment_cat'] = bitcoin_price_sentiment['sentiment'].astype('category')
    bitcoin_price_sentiment['sentiment_cat'] = bitcoin_price_sentiment['sentiment_cat'].cat.codes
    return bitcoin_price_sentiment


bitcoin_price_sentiment  = merge_bitcoin_price_and_tweets_data(bitcoin_price_df, bitcoin_tweets_sentiment_df)
bitcoin_price_sentiment['sentiment_scaled'] = bitcoin_price_sentiment['sentiment'] * 10000
bitcoin_price_sentiment = make_sentiment_column_categorical(bitcoin_price_sentiment)
bitcoin_price_sentiment

In [ ]:
def plot_two_lines(line1, line2, label1=None, label2=None, title='', xlabel=None, ylabel=None, legend_loc='best', lw=2):
    fig, ax = plt.subplots(1, figsize=(13, 7))
    ax.plot(line1, label=label1, linewidth=lw)
    ax.plot(line2, label=label2, linewidth=lw)
    ax.set_xlabel(xlabel, fontsize=14)
    ax.set_ylabel(ylabel, fontsize=14)
    ax.set_title(title, fontsize=16)
    ax.legend(loc=legend_loc, fontsize=16)

def plot_one_line(line1, label1=None, title='', xlabel=None, ylabel=None, legend_loc='best', lw=2):
    fig, ax = plt.subplots(1, figsize=(13, 7))
    ax.plot(line1, label=label1, linewidth=lw)
    ax.set_xlabel(xlabel, fontsize=14)
    ax.set_ylabel(ylabel, fontsize=14)
    ax.set_title(title, fontsize=16)
    ax.legend(loc=legend_loc, fontsize=16)

In [ ]:
plot_one_line(bitcoin_price_sentiment['Price'], 'Price', title='Analysing bitcoin price ')
plot_one_line(bitcoin_price_sentiment['sentiment'] , 'sentiment', title='Analysing tweets sentiments')
plot_one_line(bitcoin_price_sentiment['sentiment_scaled'] , 'sentiment_scaled', title='Analysing tweets sentiments')
plot_one_line(bitcoin_price_sentiment['sentiment_cat'] , 'sentiment_cat', title='Analysing tweets sentiments')

In [ ]:
plot_two_lines(bitcoin_price_sentiment['Price'], bitcoin_price_sentiment['sentiment'], 'Price', 'sentiment', title='Analysing tweets sentiments impact on bitcoin price ')
plot_two_lines(bitcoin_price_sentiment['Price'], bitcoin_price_sentiment['sentiment_scaled'], 'Price', 'sentiment_scaled', title='Analysing tweets sentiments impact on bitcoin price ')
plot_two_lines(bitcoin_price_sentiment['Price'], bitcoin_price_sentiment['sentiment_cat'], 'Price', 'sentiment_cat', title='Analysing tweets sentiments impact on bitcoin price ')

In [ ]:
bitcoin_price_sentiment['sentiment_cat_scaled'] = bitcoin_price_sentiment['sentiment_cat'] * 130
plot_two_lines(bitcoin_price_sentiment['Price'], bitcoin_price_sentiment['sentiment_cat_scaled'], 'Price', 'sentiment_cat_scaled', title='Analysing tweets sentiments impact on bitcoin price ')

In [ ]:
bitcoin_price_sentiment

In [ ]:
print("\n Cleaned tweets count = ", bitcoin_tweets_sentiment_df.sentiment.count())
print("\n Cleaned bitcoin_price count = ", bitcoin_price_df.Price.count())
print("\n Cleaned merged df count = ", bitcoin_price_sentiment.Price.count())

def get_tweet_sentiment(tweet):
    tweet_polarity = analyse_tweet(tweet)
    # set sentiment
    if tweet_polarity > 0:
        return 'positive'
    elif tweet_polarity == 0:
        return 'neutral'
    else:
        return 'negative'
    
    